In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_PATH = "ihca_cleaned.xlsx"



In [ ]:
df = pd.read_excel(DATA_PATH)
df.columns = df.columns.str.lower().str.strip()

print("Shape:", df.shape)
df.head()


Shape: (873, 28)


,mrn,gender,birth_year,arrest_year,age,coronary_artery_disease,heart_failure,heart_disease,hypertension,copd,...,survival_24h,survival_to_discharge,origin,cpr_dt,rosc_dt,arrest_dt,cpr_duration,arrest_to_cpr,shock_count,max_energy
0,3143444,M,1967.0,2018,51.0,N,N,N,Y,N,...,Y,Y,Ward,2018-06-26 15:15:00,2018-06-26 15:30:00,2018-06-26 15:15:00,-0.489583,0.000000,0,0
1,3151161,F,1963.0,2018,55.0,N,N,N,N,N,...,N,N,ICU,2018-07-14 14:32:00,2018-07-14 15:10:00,2018-07-14 14:30:00,0.026389,0.001389,0,0
2,3151503,F,1965.0,2018,53.0,N,N,N,Y,N,...,N,N,Ward,2018-07-30 15:30:00,2018-07-30 15:30:00,2018-07-30 15:30:00,0.000000,0.000000,0,0
3,3193356,F,1940.0,2018,78.0,N,N,N,Y,N,...,Y,Y,ICU,2018-10-14 14:00:00,NaT,2018-10-14 14:00:00,NaN,0.000000,0,0
4,3197363,M,1944.0,2018,74.0,N,N,N,Y,N,...,N,N,Ward,2018-06-11 04:15:00,2018-11-06 04:30:00,2018-06-11 04:15:00,148.010417,0.000000,0,0


## ENCODING SECTION

List All Y/N Columns

In [ ]:
binary_cols = [
    'coronary_artery_disease',
    'heart_failure',
    'heart_disease',
    'hypertension',
    'copd',
    'diabetes',
    'cancer',
    'covid_on_admission',
    'rosc',
    'survival_24h',
    'survival_to_discharge',
    'smoking'   # now only Y/N/U
]


In [ ]:
binary_yes_no = [
    'coronary_artery_disease',
    'heart_failure',
    'heart_disease',
    'hypertension',
    'copd',
    'diabetes',
    'cancer',
    'covid_on_admission',
    'rosc',
    'survival_24h',
    'survival_to_discharge'
]


In [ ]:
yes_no_map = {'Y': 1, 'N': 0, 'U':-1}

df[binary_yes_no] = df[binary_yes_no].replace(yes_no_map)


In [ ]:
# Smoking contains:
# Y → 1
# N → 0
# U → keep as separate category
# To encode:
df['smoking'] = df['smoking'].replace({'Y': 1, 'N': 0, 'U': -1})

In [ ]:
df['gender'] = df['gender'].map({'M': 1, 'F': 0, 'U':-1})

### Shockable rhythm flag

In [ ]:
shockable = ['VF', 'VT']

df['shockable_rhythm'] = df['initial_rhythm'].isin(shockable).astype(int)

In [ ]:
# One-Hot Encode Initial Rhythm ===
rhythm_dummies = pd.get_dummies(
    df['initial_rhythm'],
    prefix='rhythm',
    dummy_na=True,
    dtype=int
)
df = pd.concat([df.drop(columns=['initial_rhythm']), rhythm_dummies], axis=1)

In [ ]:
# === 6) One-Hot Encode Event Location ===
event_loc_dummies = pd.get_dummies(
    df['event_location'],
    prefix='loc',
    dtype=int
)

In [ ]:
# df = pd.concat([df.drop(columns=['event_location']), event_loc_dummies], axis=1)

In [ ]:
# Origin frequently conflicts with event_location (ICU origin but event in CCU, etc.)
df = df.drop(columns=['origin'])

In [ ]:
# === 8) Shock Count & Max Energy → numeric ===
df['shock_count'] = pd.to_numeric(df['shock_count'], errors='coerce')
df['max_energy']  = pd.to_numeric(df['max_energy'],  errors='coerce')

In [ ]:
# === 9) Datetime columns & clean durations ===
df['cpr_dt']    = pd.to_datetime(df['cpr_dt'])
df['rosc_dt']   = pd.to_datetime(df['rosc_dt'])
df['arrest_dt'] = pd.to_datetime(df['arrest_dt'])

In [ ]:
# === 9) Datetime columns & clean durations ===
df['cpr_duration_min'] = (df['rosc_dt'] - df['cpr_dt']).dt.total_seconds() / 60
df['arrest_to_cpr_min'] = (df['cpr_dt'] - df['arrest_dt']).dt.total_seconds() / 60


In [ ]:
for col in ['cpr_duration', 'arrest_to_cpr']:
    if col in df.columns:
        df = df.drop(columns=[col])

In [ ]:
encoding_info = {
    "binary_yes_no": binary_yes_no,
    "binary_yes_no_map": yes_no_map,
    "smoking_encoding": {"Y": 1, "N": 0, "U": -1},
    "gender_encoding": {"M": 1, "F": 0},
    "shockable_rhythm": {"VF/VT": 1, "other": 0},
    "initial_rhythm_dummies": list(rhythm_dummies.columns),
    "event_location_dummies": list(event_loc_dummies.columns),
    "duration_columns": ["cpr_duration_min", "arrest_to_cpr_min"],
}

encoding_info

{'binary_yes_no': ['coronary_artery_disease',
  'heart_failure',
  'heart_disease',
  'hypertension',
  'copd',
  'diabetes',
  'cancer',
  'covid_on_admission',
  'rosc',
  'survival_24h',
  'survival_to_discharge'],
 'binary_yes_no_map': {'Y': 1, 'N': 0, 'U': -1},
 'smoking_encoding': {'Y': 1, 'N': 0, 'U': -1},
 'gender_encoding': {'M': 1, 'F': 0},
 'shockable_rhythm': {'VF/VT': 1, 'other': 0},
 'initial_rhythm_dummies': ['rhythm_AF',
  'rhythm_Asystole',
  'rhythm_Bradycardia',
  'rhythm_PEA',
  'rhythm_Sinus/Other',
  'rhythm_VF',
  'rhythm_VT',
  'rhythm_nan'],
 'event_location_dummies': ['loc_CCU',
  'loc_COVID_WARD',
  'loc_CVU',
  'loc_DSU',
  'loc_FMS',
  'loc_ICU',
  'loc_MMS',
  'loc_NEONATAL_ICU',
  'loc_OBSTETRICS',
  'loc_OPERATING_ROOM',
  'loc_PEDIATRIC',
  'loc_PEDIATRIC_ICU',
  'loc_POST_ICU',
  'loc_UNKNOWN'],
 'duration_columns': ['cpr_duration_min', 'arrest_to_cpr_min']}

In [ ]:
# we want for EDA this dataframe
# Columns we *explicitly* keep
base_eda_cols = [
    # Demographics
    "age", "gender", "smoking",

    # Comorbidities
    "coronary_artery_disease",
    "heart_failure",
    "heart_disease",
    "hypertension",
    "copd",
    "diabetes",
    "cancer",
    "covid_on_admission",

    # Arrest characteristics
    "shock_count",
    "max_energy",
    "shockable_rhythm",

    # Outcomes
    "rosc",
    "survival_24h",
    "survival_to_discharge",

    # Timing
    "cpr_duration_min",
    "arrest_to_cpr_min",
    "cpr_dt",
    "arrest_dt",
    "rosc_dt",

    # rythm
    "initial_rhythm",
    "event_location",
]

# Add all rhythm_* and loc_* dummies automatically
rhythm_cols = [c for c in df.columns if c.startswith("rhythm_")]
loc_cols    = [c for c in df.columns if c.startswith("loc_")]

eda_cols = base_eda_cols + rhythm_cols + loc_cols
# print encoding rules
print("Encoding Rules:")
for col in eda_cols:
    if col in df.columns:
        print(f"{col}: {df[col].dtype}")




# Final EDA dataframe
df_eda = df[eda_cols].copy()



Encoding Rules:
age: float64
gender: int64
smoking: object
coronary_artery_disease: object
heart_failure: object
heart_disease: object
hypertension: object
copd: object
diabetes: object
cancer: object
covid_on_admission: object
shock_count: int64
max_energy: int64
shockable_rhythm: int64
rosc: object
survival_24h: object
survival_to_discharge: object
cpr_duration_min: float64
arrest_to_cpr_min: float64
cpr_dt: datetime64[ns]
arrest_dt: datetime64[ns]
rosc_dt: datetime64[ns]
rhythm_AF: int64
rhythm_Asystole: int64
rhythm_Bradycardia: int64
rhythm_PEA: int64
rhythm_Sinus/Other: int64
rhythm_VF: int64
rhythm_VT: int64
rhythm_nan: int64
loc_CCU: int64
loc_COVID_WARD: int64
loc_CVU: int64
loc_DSU: int64
loc_FMS: int64
loc_ICU: int64
loc_MMS: int64
loc_NEONATAL_ICU: int64
loc_OBSTETRICS: int64
loc_OPERATING_ROOM: int64
loc_PEDIATRIC: int64
loc_PEDIATRIC_ICU: int64
loc_POST_ICU: int64
loc_UNKNOWN: int64


KeyError: "['initial_rhythm', 'event_location'] not in index"

In [ ]:
df_eda.head(5)

,age,gender,smoking,coronary_artery_disease,heart_failure,heart_disease,hypertension,copd,diabetes,cancer,...,loc_FMS,loc_ICU,loc_MMS,loc_NEONATAL_ICU,loc_OBSTETRICS,loc_OPERATING_ROOM,loc_PEDIATRIC,loc_PEDIATRIC_ICU,loc_POST_ICU,loc_UNKNOWN
0,51.0,1,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,55.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53.0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,78.0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,74.0,1,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
# export
df_eda.to_csv("ihca_encoded_eda.csv", index=False)